In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)
import os
os.chdir('..')

# 内存保存历史 + 窗口记忆

In [2]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

model = ChatOpenAI()
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一个数学老师。"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)
chain = prompt | model | StrOutputParser()

In [3]:
from langchain_chinese import MemoryManager
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory

window = ConversationBufferWindowMemory(
  return_messages=True, k=2, chat_memory = ChatMessageHistory()
)

memory = MemoryManager(shorterm_memory = window)

In [4]:
from langchain_chinese import WithMemoryBinding

withMemoryChain = WithMemoryBinding(
  chain,
  memory,
  input_messages_key="input",
  history_messages_key="history",
)

In [5]:
withMemoryChain.invoke(
  {"input": "三角函数什么意思？?"},
  config={"configurable": {"session_id": "abc123"}},
)

'三角函数是指在直角三角形中，角的大小关系到三角形的边长比例的函数。常见的三角函数包括正弦函数、余弦函数、正切函数、余切函数、正割函数和余割函数等。这些函数在数学和物理中有着广泛的应用，可以描述角度和三角形之间的关系。'

In [8]:
withMemoryChain.invoke(
  {"input": "正弦是什么?"},
  config={"configurable": {"session_id": "abc123"}},
)

'在一个直角三角形中，正弦是一个角的对边长度与斜边长度的比值。正弦函数通常用sin表示，对于一个角θ而言，其正弦值可以表示为：sin(θ) = 对边 / 斜边。正弦函数在三角学和数学中有着重要的应用，可以帮助我们计算角度和边长之间的关系。'

In [9]:
withMemoryChain.invoke(
  {"input": "小学会学到吗?"},
  config={"configurable": {"session_id": "abc123"}},
)

'正弦函数通常不是小学阶段的数学内容，因为它涉及到三角函数和三角学的概念，通常在初中或高中阶段学习。小学阶段的数学教育主要集中在基本的数学运算、几何图形、分数、小数等方面，正弦函数等高级数学概念一般在更高年级的学习中才会接触到。'

In [11]:
memory.get_shorterm_memory("abc123").buffer_as_messages

[HumanMessage(content='正弦是什么?'),
 AIMessage(content='在一个直角三角形中，正弦是一个角的对边长度与斜边长度的比值。正弦函数通常用sin表示，对于一个角θ而言，其正弦值可以表示为：sin(θ) = 对边 / 斜边。正弦函数在三角学和数学中有着重要的应用，可以帮助我们计算角度和边长之间的关系。'),
 HumanMessage(content='小学会学到吗?'),
 AIMessage(content='正弦函数通常不是小学阶段的数学内容，因为它涉及到三角函数和三角学的概念，通常在初中或高中阶段学习。小学阶段的数学教育主要集中在基本的数学运算、几何图形、分数、小数等方面，正弦函数等高级数学概念一般在更高年级的学习中才会接触到。')]

In [12]:
memory.get_shorterm_memory("abc123").chat_memory.messages

[HumanMessage(content='三角函数什么意思？?'),
 AIMessage(content='三角函数是指在直角三角形中，角的大小关系到三角形的边长比例的函数。常见的三角函数包括正弦函数、余弦函数、正切函数、余切函数、正割函数和余割函数等。这些函数在数学和物理中有着广泛的应用，可以描述角度和三角形之间的关系。'),
 HumanMessage(content='正弦是什么?'),
 AIMessage(content='在一个直角三角形中，正弦是一个角的对边长度与斜边长度的比值。正弦函数通常用sin表示，对于一个角θ而言，其正弦值可以表示为：sin(θ) = 对边 / 斜边。正弦函数在三角学和数学中有着重要的应用，可以帮助我们计算角度和边长之间的关系。'),
 HumanMessage(content='小学会学到吗?'),
 AIMessage(content='正弦函数通常不是小学阶段的数学内容，因为它涉及到三角函数和三角学的概念，通常在初中或高中阶段学习。小学阶段的数学教育主要集中在基本的数学运算、几何图形、分数、小数等方面，正弦函数等高级数学概念一般在更高年级的学习中才会接触到。')]

In [27]:
for chunk in withMemoryChain.stream(
  {"input": "那小学数学学什么?"},
  config={"configurable": {"session_id": "abc123"}},
):
  print(chunk, end="|", flush=True)

|小|学|阶|段|的|数|学|教|育|主|要|围|绕|建|立|学|生|的|数|学|基|础|，|培|养|其|数|学|思|维|和|解|决|问题|的|能|力|。|一|般|来|说|，|小|学|数|学|的|学|习|内容|包|括|但|不|限|于|以下|几|个|方|面|：

|1|.| 数|的|认|识|和|运|算|：|包|括|整|数|、|分|数|、|小|数|等|的|认|识|与|四|则|运|算|。
|2|.| 几|何|图|形|：|学|习|认|识|各|种|几|何|图|形|，|如|直|线|、|角|、|三|角|形|、|四|边|形|等|，并|学|习|测|量|、|比|较|和|计|算|这|些|几|何|图|形|的|性|质|。
|3|.| 时间|和|日|历|：|学|习|认|识|时间|的|概|念|、|钟|表|读|数|、|日期|的|表|达|等|。
|4|.| 数据|与|统|计|：|学|习|收|集|数据|、|整|理|数据|、|制|作|简|单|图|表|、|进行|数据|分|析|等|。
|5|.| |逻|辑|推|理|：|学|习|简|单|的|逻|辑|推|理|和|解|决|问题|的|方法|。
|6|.| 口|算|和|应|用|题|：|通过|日|常|生|活|中|的|实|际|问题|，|培|养|学|生|的|数|学|思|维|和|解|决|问题|的|能|力|。

|小|学|数|学|教|育|的|目|标|是|帮|助|学|生|建|立|扎|实|的|数|学|基|础|，|打|下|良|好|的|数|学|基|础|，|为|进|一|步|学|习|更|高|级|的|数|学|知|识|打|下|基|础|。||

# redis持久化

```shell
pip install -U langchain-community redis
```

创建窗口管理器时要指定redis的工厂函数，其他与上面的例子相同。

In [ ]:
from langchain_chinese import MemoryManager
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory
from langchain_community.chat_message_histories import RedisChatMessageHistory

window = ConversationBufferWindowMemory(
    return_messages=True, k=2, chat_memory = ChatMessageHistory()
)

memory = MemoryManager(
    lambda session_id: RedisChatMessageHistory(
        session_id, url="redis://localhost:6379"
    ),
    shorterm_memory = window
)